卷积神经网络之损失函数(Pytorch版)
===

# 1.L1Loss:MAELoss-mean absolute error 
# 2.MSELoss: mean squared error Loss

# 3.CrossEntropyLoss
多分类用的交叉熵损失函数，在Pytorch里，它就相当于LogSoftMax与NLLLoss的合体。我在代码[Code](../ELib/pyn/cnn/layers.py)里面实现的Softmax就是这个损失函数。

这个损失函数用于多分类问题，它得到的结果是每个分类的条件概率。我们知道，监督学习解决的问题就是通过优化相应的目标函数使得输出的这个输出概率分布能够逼近真实的概率分布。所以这里的目标函数就是用于描述生成的概率分布与真实概率分布的（负的相似度），目标函数的值越小，生成的概率分布与真实概率分布就越相似

## 3.1.Softmax介绍
$$\sigma_i(Z)=\frac{e^{z_i}}{\sum_{k=1}^Ke^{z_k}}, j=1,2,...,K$$
![image](Images/03/00/05_09_001.jpg)

In [2]:
import numpy as np
prediction = np.array([0.02760797,-0.00083022,0.00351067,-0.00328555,0.00372311,-0.00814351,-0.00122638,-0.00717432,0.00035223,0.00667405])
prediction = np.reshape(prediction, newshape=(1, 10))
exp_prediction = np.zeros(prediction.shape)
softmax = np.zeros(prediction.shape)
for i in range(prediction.shape[0]):
    prediction[i, :] -= np.max(prediction[i, :])
    exp_prediction[i] = np.exp(prediction[i])
    softmax[i] = exp_prediction[i]/np.sum(exp_prediction[i])
print(softmax)

[[0.10257674 0.09970073 0.10013446 0.09945623 0.10015573 0.09897425
  0.09966124 0.09907022 0.09981869 0.10045172]]


上述算法实现了Softmax的过程。prediction是神经网络预测出来的当前图片所对应的分类的概率。我们将prediction里面的每个值都减去分类概率的最大值，这样并不会影响最终的结果，但是却能使得我们计算exp时遇到上下溢的情况后表现的更为正常。

这是因为对一个比较大的数求exponential非常容易发生overflow。求exponential之前将$z$的每一个元素减去$z_i$的最大值。这样求exponential的时候会碰到的最大的数就是0了，不会发生overflow的问题，但是如果其他数原本是正常范围，现在全部被减去了一个非常大的数，于是都变成了绝对值非常大的负数，所以全部都会发生underflow，但是underflow的时候得到的是0，这其实是非常meaningful的近似值，而且后续的计算也不会出现奇怪的NaN.最关键的是最后我们一般都会做normalization，所以同时减去一个任意数是不会改变最终的预测的


## 3.2.Cross entropy 交叉熵损失-用于分类问题
对Softmax去最大似然，有:
$$log(\sum_y(z))=log(\frac{e^{z_y}}{\sum_{j=1}^me^{z_j}})=z_y-log(\sum_{j=1}^me^{z_j})$$
习惯上一般都是优化到最小值，所以这里我们的loss=-likehood.

In [ ]:
import numpy as np
loss = 0
label = 8
for i in range(prediction.shape[0]):
    loss += np.log(np.sum(np.exp(prediction[i]))) - prediction[i, label]

## 3.3.反向传播
Loss是反向传播的起点

In [ ]:
eta = softmax.copy()
batch_size=1
label = 8
for i in range(batch_size):
    eta[i, 8] -= 1

eta就是$$\frac{\partial{loss}}{\partial{convout}}$$。根据[链式求导法则](03.深度学习.00.神经网络基础.04.链式法则与反向传播算法.ipynb)里面介绍的，每一层的反向传播算法都需要做两件事儿
- 根据传入的$\frac{\partial{loss}}{\partial{x_1}}$求解出$\frac{\partial{loss}}{\partial{x}}$
- 求出$\frac{\partial{x_1}}{\partial{W_1}}$和$\frac{\partial{x_1}}{\partial{b_1}}$

求出第一个值，是为了将这个反传给上一层，以此保证导数可以流动起来。求出第二个，则是为了更新当前层的两个参数。损失函数是没有参数的，所以只需要求第一个就好了。对于第一个值来说，有
$$
\begin{split}
\frac{\partial{loss}}{\partial{z_k}}&=\frac{\partial{log(\sum_{j=1}^me^{z_j})-z_y}}{\partial{z_k}} \\
&=\frac{1}{\sum_{j=1}^me^{z_j}} \times \frac{\partial{sum_{j=1}^me^{z_j}}}{\partial{z_k}} - \frac{\partial{z_y}}{\partial{z_k}} \\
&=\frac{1}{\sum_{j=1}^me^{z_j}} \times \frac{\partial{e^{z_1}+e^{z_2}+...+e^{z_k}+...+e^{z_j}}}{\partial{z_k}}-\frac{\partial{z_y}}{\partial{z_k}} \\
&=\frac{1}{\sum_{j=1}^me^{z_j}} \times e^{z_k}-\frac{\partial{z_y}}{\partial{z_k}} \\
&=\frac{e^{z_k}}{\sum_{j=1}^me^{z_j}}-\frac{\partial{z_y}}{\partial{z_k}}
\end{split}
$$

我们可以很清楚的看到，第一部分就是softmax，也就是eta。那第二部分是什么呢，如果$k=y$那么结果就是1，否则就是0.也就是我们上面算式的写法。

# 8.BCELoss:Binary Cross Entropy Loss
# 9.BCEWithLogitsLoss:
# 13.SmoothL1Loss
